In [1]:
# pretty tracebacks
%load_ext rich

# https://rich.readthedocs.io/en/stable/reference/init.html?#rich.inspect
from rich import inspect

### Function Arguments

In [3]:
# 0). positional-only argument  # Python 3.8 PEP 570 -- https://www.python.org/dev/peps/pep-0570
# 1). positional or keyword argument
# 2). positional or keyword argument with default value
# 3). variable positional arguments
# 4). keyword-only argument
# 5). keyword-only argument with default value
# 6). variable keyword-only arguments

def func_args(positional_only_parameters,
              /,
              positional_or_keyword_parameters,
              positional_or_keyword_parameters_w_default='default',
              *w,
              keyword_only_parameters,
              keyword_only_parameters_w_default='default',
              **kw):
    print(f"positional_only_parameters={positional_only_parameters},\n"
          f"positional_or_keyword_parameters={positional_or_keyword_parameters},\n"
          f"positional_or_keyword_parameters_w_default={positional_or_keyword_parameters_w_default},\n"
          f"w={w},\n"
          f"keyword_only_parameters={keyword_only_parameters},\n"
          f"keyword_only_parameters_w_default={keyword_only_parameters_w_default},\n"
          f"kw={kw}")


In [104]:
func_args(1,2,3,4,5,6,keyword_only_parameters=4)

positional_only_parameters=1,
positional_or_keyword_parameters=2,
positional_or_keyword_parameters_w_default=3,
w=(4, 5, 6),
keyword_only_parameters=4,
keyword_only_parameters_w_default=default,
kw={}


In [105]:
func_args(0, 1, keyword_only_parameters=3)

positional_only_parameters=0,
positional_or_keyword_parameters=1,
positional_or_keyword_parameters_w_default=default,
w=(),
keyword_only_parameters=3,
keyword_only_parameters_w_default=default,
kw={}


In [108]:
func_args(0, positional_or_keyword_parameters_w_default=2, positional_or_keyword_parameters=1, keyword_only_parameters_w_default=4, keyword_only_parameters=3)

positional_only_parameters=0,
positional_or_keyword_parameters=1,
positional_or_keyword_parameters_w_default=2,
w=(),
keyword_only_parameters=3,
keyword_only_parameters_w_default=4,
kw={}


----

### Metaprogramming

#### Class Statement Protocol

Technically speaking, Python follows a standard protocol to make this happen: at the end of a `class` statement, and after running all its nested code in a namespace dictionary corresponding to the class’s local scope, Python calls the `type` object to create the `class` object like this:
```
    classname = type(classname, superclasses, attributedict)
```
> Note that [`x(arg1, arg2, ...)` roughly translates to `type(x).__call__(x, arg1, ...)`](https://docs.python.org/3/reference/datamodel.html#emulating-callable-objects). So here it will invoke `type.__call__(...)`.

The `type` object defines a `__call__` operator overloading method that runs two other methods when the `type` object is called:
```
    type.__new__(typeclass, classname, superclasses, attributedict)
    type.__init__(class, classname, superclasses, attributedict)
```

The `__new__` method creates and returns the new `class` object, and then the `__init__` method initializes the newly created object. These are the hooks that metaclass subclasses of `type` generally use to customize classes.

For example, given a class definition like the following for Spam:

In [109]:
class Eggs: ...                  # Inherited names here

class Spam(Eggs):                # Inherits from Eggs
    data = 1                     # Class data attribute
    def meth(self, arg):         # Class method attribute
        return self.data + arg

Python will internally run the nested code block to create two attributes of the class (data and meth), and then call the type object to generate the class object at the end of the class statement:

In [71]:
Spam = type('Spam', (Eggs,), {'data': 1, 'meth': Spam.meth, '__module__': '__main__'})

In [62]:
type(type), type(object)

(type, type)

In [63]:
type.__class__, object.__class__

(type, type)

In [64]:
isinstance(type, object), isinstance(object, object)

(True, True)

#### Overloading class creation calls with metaclasses
> Learning Python 40. Metaclasses : Coding Metaclass

In [1]:
class SuperMeta(type):
    def __call__(meta, classname, supers, classdict):
        print('In SuperMeta.call: ', classname, supers, classdict, sep='\n...')
        return type.__call__(meta, classname, supers, classdict)
    def __new__(meta, classname, supers, classdict):
        print('In SuperMeta.new: ', classname, supers, classdict, sep='\n...')
        return type.__new__(meta, classname, supers, classdict)
    def __init__(Class, classname, supers, classdict):
        print('In SuperMeta init:', classname, supers, classdict, sep='\n...')
        print('...init class object:', list(Class.__dict__.keys()))

print('making metaclass')
class SubMeta(type, metaclass=SuperMeta):
    def __new__(meta, classname, supers, classdict):
        print('In SubMeta.new: ', classname, supers, classdict, sep='\n...')
        return type.__new__(meta, classname, supers, classdict)
    def __init__(Class, classname, supers, classdict):
        print('In SubMeta init:', classname, supers, classdict, sep='\n...')
        print('...init class object:', list(Class.__dict__.keys()))

class Eggs:
    pass

print('making class')
class Spam(Eggs, metaclass=SubMeta): # Invoke SubMeta, via SuperMeta.__call__
    data = 1
    def meth(self, arg):
        return self.data + arg

print('making instance')
X = Spam()
print('data:', X.data, X.meth(2))

making metaclass
In SuperMeta.new: 
...SubMeta
...(<class 'type'>,)
...{'__module__': '__main__', '__qualname__': 'SubMeta', '__new__': <function SubMeta.__new__ at 0x7fffe9dc6430>, '__init__': <function SubMeta.__init__ at 0x7fffe9dc64c0>}
In SuperMeta init:
...SubMeta
...(<class 'type'>,)
...{'__module__': '__main__', '__qualname__': 'SubMeta', '__new__': <function SubMeta.__new__ at 0x7fffe9dc6430>, '__init__': <function SubMeta.__init__ at 0x7fffe9dc64c0>}
...init class object: ['__module__', '__new__', '__init__', '__doc__']
making class
In SuperMeta.call: 
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x7fffe9dd21f0>}
In SubMeta.new: 
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x7fffe9dd21f0>}
In SubMeta init:
...Spam
...(<class '__main__.Eggs'>,)
...{'__module__': '__main__', '__qualname__': 'Spam', 'da

In [ ]:
type(SuperMeta)

In [2]:
class A:
    class B:
        class C:
            def testfunc():
                pass

dir(A.B.C.testfunc)
print(A.B.C.testfunc.__qualname__)
global_testfunc = A.B.C.testfunc
print(global_testfunc.__qualname__)

A.B.C.testfunc
A.B.C.testfunc


In [ ]:
import inspect
# inspect.getmembers(A.B.C.testfunc)

In [35]:
type(A.B.C.testfunc)

function

In [36]:
issubclass(type(A.B.C.testfunc), object)

True

In [69]:
dir(A.B.C.testfunc)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

----

#### Descriptor

In [72]:
#####################
# Descriptor usage #
#####################

# https://docs.python.org/3.7/howto/descriptor.html#descriptor-example

class RevealAccess(object):
    """A data descriptor that sets and returns values
       normally and prints a message logging their access.
    """

    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name

    def __get__(self, obj, objtype):
        print('Retrieving', self.name)
        return self.val
    
    def __set__(self, obj, val):
        print('Updating', self.name)
        self.val = val

class MyClass(object):
    x = RevealAccess(10, 'var "x"')
    y = 5
    
    def __getattribute__(self, name):
        print ("__getattribute__ ==> self:" + repr(self) + " name:" + str(name))
        if name != "__dict__" and name in self.__dict__:
            print("get defined property: " + str(name))
            return self.__dict__[name]
        else:
            print("get property: " + str(name))
            ############################################################################################
            # !!!! The object.__getattribute__(...) will handle the invoking of __get__() method. !!!! #
            ############################################################################################
            return super().__getattribute__(name)

    def __setattr__(self, name, value):
        if name in self.__dict__:
            print("set defined property: " + str(name))
            self.__dict__[name] = value
        else:
            print("set property: " + str(name))
            #######################################################################################
            # !!!! The object.__setattr__(...) will handle the invoking of __set__() method. !!!! #
            #######################################################################################
            super().__setattr__(name, value)


m = MyClass()
m.x
print("=" * 80)
m.y
print("=" * 80)
m.x = 1

__getattribute__ ==> self:<__main__.MyClass object at 0x7fffe9e34640> name:x
__getattribute__ ==> self:<__main__.MyClass object at 0x7fffe9e34640> name:__dict__
get property: __dict__
get property: x
Retrieving var "x"
__getattribute__ ==> self:<__main__.MyClass object at 0x7fffe9e34640> name:y
__getattribute__ ==> self:<__main__.MyClass object at 0x7fffe9e34640> name:__dict__
get property: __dict__
get property: y
__getattribute__ ==> self:<__main__.MyClass object at 0x7fffe9e34640> name:__dict__
get property: __dict__
set property: x
Updating var "x"


#### `__class__` & `__dict__` attributes

The special `__class__` and `__dict__` attributes are data descriptor. It is used to ensure that the special __class__ and __dict__ attributes cannot be redefined by the same names in an instance’s own __dict__:
> Learning Python 40. Metaclasses : Inheritance and Instance, The descriptors special case

In [83]:
class C: pass                          # Inheritance special case #1...
I = C()                                # Class data descriptors have precedence
I.__class__, I.__dict__

(__main__.C, {})

In [89]:
I.__dict__['name'] = 'bob'             # Dynamic data in the instance
I.__dict__['__class__'] = 'spam'       # Assign keys, not attributes
I.__dict__['__dict__']  = {}

In [85]:
I.name                                 # I.name comes from I.__dict__ as usual
## 'bob'                               # But I.__class__ and I.__dict__ do not!

'bob'

In [87]:
I.__class__, I.__dict__
## (<class '__main__.C'>, {'__class__': 'spam', '__dict__': {}, 'name': 'bob'})

(__main__.C, {'name': 'bob', '__class__': 'spam', '__dict__': {}})

##### Property Implementation

In [90]:
# https://docs.python.org/3.10/howto/descriptor.html#properties

class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc
        self._name = ''

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError(f'unreadable attribute {self._name}')
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError(f"can't set attribute {self._name}")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError(f"can't delete attribute {self._name}")
        self.fdel(obj)

    #def getter(self, fget):
    #    prop = type(self)(fget, self.fset, self.fdel, self.__doc__)
    #    prop._name = self._name
    #    return prop
    def getter(self, fget):
        self.fget = fget
        return self

    #def setter(self, fset):
    #    prop = type(self)(self.fget, fset, self.fdel, self.__doc__)
    #    prop._name = self._name
    #    return prop
    def setter(self, fset):
        self.fset = fset
        return self

    def deleter(self, fdel):
        prop = type(self)(self.fget, self.fset, fdel, self.__doc__)
        prop._name = self._name
        return prop

In [7]:
class Box:
    bird = Property()
    
    @bird.getter
    def get_bird(self):
        return self._bird

    @bird.setter
    def set_bird(self, value):
        self._bird = value

In [8]:
box = Box()
box.bird = 10
box.bird

10

----

### Attribute Access (`__getattribute__` demystify)

####  `object.__getattribute__() pure Python equivalent`

In [70]:
# https://docs.python.org/3.10/howto/descriptor.html#invocation-from-an-instance

def object_getattribute(obj, name):
    "Emulate PyObject_GenericGetAttr() in Objects/object.c"
    null = object()
    objtype = type(obj)
    cls_var = getattr(objtype, name, null)
    descr_get = getattr(type(cls_var), '__get__', null)
    if descr_get is not null:
        if (hasattr(type(cls_var), '__set__')
            or hasattr(type(cls_var), '__delete__')):
            return descr_get(cls_var, obj, objtype)     # data descriptor
    if hasattr(obj, '__dict__') and name in vars(obj):
        return vars(obj)[name]                          # instance variable
    if descr_get is not null:
        return descr_get(cls_var, obj, objtype)         # non-data descriptor
    if cls_var is not null:
        return cls_var                                  # class variable
    raise AttributeError(name)

**`Note, there is no __getattr__() hook in the __getattribute__() code.`** `That is why calling __getattribute__() directly or with super().__getattribute__() will bypass __getattr__() entirely.
Instead, it is the dot operator and the getattr() function that are responsible for invoking __getattr__() whenever __getattribute__() raises an AttributeError. Their logic is encapsulated in a helper function:`

In [68]:
def getattr_hook(obj, name):
    "Emulate slot_tp_getattr_hook() in Objects/typeobject.c"
    try:
        return obj.__getattribute__(name)
    except AttributeError:
        if not hasattr(type(obj), '__getattr__'):
            raise
    return type(obj).__getattr__(obj, name)             # __getattr__

**[summary-of-invocation-logic](https://docs.python.org/3.10/howto/descriptor.html#summary-of-invocation-logic)**

#### Attribute Reference/Lookup
> Learning Python 40. Metaclasses : Inheritance and Instance

With both the data descriptor special case and general descriptor invocation factored in with class and metaclass trees, Python’s full new-style inheritance algorithm can be stated as follows—a complex procedure, which assumes knowledge of descriptors, metaclasses, and MROs, but is the final arbiter of attribute names nonetheless (in the following, items are attempted in sequence either as numbered, or per their left-to-right order in “or” conjunctions):

_To look up an explicit attribute name:_
1. From an instance I, search the instance, its class, and its superclasses, as follows:
    - a. Search the `__dict__` of all classes on the `__mro__` found at I’s `__class__`
    - b. If a data descriptor was found in step _a_, call its `__get__` and exit
    - c. Else, return a value in the `__dict__` of the instance **I**
    - d. Else, call a nondata descriptor or return a value found in step _a_
2. From a _class_ **C**, search the class, its superclasses, and its metaclasses tree, as follows:
    - a. Search the `__dict__` of all metaclasses on the `__mro__` found at **C**’s `__class__`
    - b. If a data descriptor was found in step _a_, call its `__get__` and exit
    - c. Else, call a descriptor or return a value in the `__dict__` of a class on **C**’s own `__mro__`
    - d. Else, call a nondata descriptor or return a value found in step _a_


### Assignment inheritance

When an attribute assignment is run for new-style classes, a data descriptor with a `__set__` method is acquired from a class by inheritance using the MRO, and has precedence over the normal storage model. In terms of the prior section’s rules:

 - When applied to an instance, such assignments essentially follow steps _a_ through _c_ of rule 1, searching the instance’s class tree, though step b calls `__set__` instead of `__get__`, and step c stops and stores in the instance instead of attempting a fetch.
 - When applied to a class, such assignments run the same procedure on the class’s metaclass tree: roughly the same as rule 2, but step c stops and stores in the class.

Because descriptors are also the basis for other advanced attribute tools such as properties and slots, this inheritance pre-check on assignment is utilized in multiple contexts. The net effect is that descriptors are treated as an inheritance special case in new-style classes, for both reference and assignment. 

----

#### Scope

In [9]:
# Each call to a function creates a new local scope.

def factory(init_stat=1):
    def func1(a):
        nonlocal init_stat
        print("%s init_stat=%d" % (a, init_stat))
        init_stat += 1
        
    def func2(b):
        nonlocal init_stat
        print("%d init_stat=%d" % (b, init_stat))
        init_stat += 1
    return func1, func2


funcs0 = factory()
funcs0[0]("cold")
funcs0[1](257745)

funcs1 = factory(10)
funcs1[0]("spam")
funcs1[1](245674)

funcs2 = factory(20)
funcs2[0]("duck")
funcs2[1](757899)

cold init_stat=1
257745 init_stat=2
spam init_stat=10
245674 init_stat=11
duck init_stat=20
757899 init_stat=21


In [10]:
for func_dir in dir(funcs0[0]):
    print(func_dir, end=' ')
    if callable(eval('funcs0[0].' + func_dir)):
        try:
            print(eval('funcs0[0].' + func_dir + '()'))
        except:
            print("can't call w/o args")
    else:
        print(eval('funcs0[0].' + func_dir))

__annotations__ {}
__call__ can't call w/o args
__class__ can't call w/o args
__closure__ (<cell at 0x7fffe9dd1910: int object at 0x5555558ebe20>,)
__code__ <code object func1 at 0x7fffe9dd75b0, file "/tmp/ipykernel_28080/469221007.py", line 4>
__defaults__ None
__delattr__ can't call w/o args
__dict__ {}
__dir__ ['__repr__', '__call__', '__get__', '__new__', '__closure__', '__doc__', '__globals__', '__module__', '__code__', '__defaults__', '__kwdefaults__', '__annotations__', '__dict__', '__name__', '__qualname__', '__hash__', '__str__', '__getattribute__', '__setattr__', '__delattr__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__init__', '__reduce_ex__', '__reduce__', '__subclasshook__', '__init_subclass__', '__format__', '__sizeof__', '__dir__', '__class__']
__doc__ None
__eq__ can't call w/o args
__format__ can't call w/o args
__ge__ can't call w/o args
__get__ can't call w/o args
__getattribute__ can't call w/o args
__globals__ {'__name__': '__main__', '__doc__'

In [15]:
X = 11

def f():     # Global (module) name/attribute (X, or manynames.X)
    print(X) # Access global X (11)

def g():
    X = 22
    print(X) # Local (function) variable (X, hides module X)

#################################################
#### !!! class is not an enclosing scope !!! ####
#################################################
class C:
    print(X)
    X = 33 # This should be treated as attribute
    print(X)
    def m(self):
        #################################################
        print(X)     # !!! NOT 33 !!!
        #################################################
        print(C.X)   # Class attribute (C.X)
    def m2():
        X = 44       # Local variable in method (X)
        self.X = 55  # Instance attribute (instance.X)

I = C()
I.m()


11
33
11
33


In [19]:
X = 1

def nester():
    print(X) # Global: 1
    class C:
        print(X) # Global: 1
        def method1(self):
            print(X) # Global: 1
        def method2(self):
            X = 3 # Hides global
            print(X) # Local: 3
    print()
    I = C()
    I.method1()
    I.method2()

print(X)
nester()

# Global: 1 # Rest: 1, 1, 1, 3

1
1
1

1
3
----------------------------------------


In [3]:
#### https://stackoverflow.com/questions/4296677

i = 6
def f(x):
    def g():
        print i
    # ...
    # skip to the next page
    # ...
    for i in x:  # ah, i *is* local to f, so this is what g sees
        pass
    g()
## The call to g() will refer to the variable i bound in f() by the for loop. If g() is called before the loop is executed, a NameError will be raised.

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(i)? (3888779665.py, line 4)

In [16]:
class rec: pass

rec.name = 'Bob'
rec.age = 40

print(rec.name)

x = rec()
y = rec()

print(x.name, y.name)
x.name = 'Sue'
print(rec.name, x.name, y.name)
rec.name = 'SharedName'
print(rec.name, x.name, y.name)

Bob
Bob Bob
Bob Sue Bob
SharedName Sue SharedName


In [17]:
# The class’s namespace dictionary shows the name and age attributes
# we assigned to it, x has its own name, and y is still empty.
print(list(rec.__dict__.keys()))
print(list(name for name in rec.__dict__ if not name.startswith('__')))
print(list(x.__dict__.keys()))
print(list(y.__dict__.keys()))


['__module__', '__dict__', '__weakref__', '__doc__', 'name', 'age']
['name', 'age']
['name']
[]


In [18]:
print(x.__class__)
print(rec.__bases__)
print(object.__dict__.keys())
print()

print(dir(x))
print(dir(rec))

<class '__main__.rec'>
(<class 'object'>,)
dict_keys(['__repr__', '__hash__', '__str__', '__getattribute__', '__setattr__', '__delattr__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__init__', '__new__', '__reduce_ex__', '__reduce__', '__subclasshook__', '__init_subclass__', '__format__', '__sizeof__', '__dir__', '__class__', '__doc__'])

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'age', 'name']
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr_

----

In [11]:
def my_load_mod1():
    import sys
def my_load_mod2():
    import sys
    return sys

try:
    my_load_mod1()
    print(sys.path)
except:
    print('import only make the module name available in enclosing scope')
    sys = my_load_mod2()

print(sys.path)

import only make the module name available in enclosing scope
['/home/scratch.edwardc_gpu_2/lab/data/notebook_root/mypylab', '/home/edwardc/lib/pylib', '/home/edwardc/lib/pylib/pudb-2019.2-py3.7.egg', '/home/edwardc/lib/pylib/urwid-2.1.0-py3.7-linux-x86_64.egg', '/home/scratch.edwardc_gpu_2/lab/tools/anaconda3/envs/d2l/lib/python38.zip', '/home/scratch.edwardc_gpu_2/lab/tools/anaconda3/envs/d2l/lib/python3.8', '/home/scratch.edwardc_gpu_2/lab/tools/anaconda3/envs/d2l/lib/python3.8/lib-dynload', '', '/home/edwardc/.local/lib/python3.8/site-packages', '/home/scratch.edwardc_gpu_2/lab/tools/anaconda3/envs/d2l/lib/python3.8/site-packages', '/home/scratch.edwardc_gpu_2/lab/tools/anaconda3/envs/d2l/lib/python3.8/site-packages/IPython/extensions', '/home/edwardc/.ipython']


----

#### Module loading

In [20]:
# module search path

import sys
print(sys.path)

['/home/scratch.edwardc_gpu_2/lab/data/notebook_root/mypylab', '/home/edwardc/lib/pylib', '/home/edwardc/lib/pylib/pudb-2019.2-py3.7.egg', '/home/edwardc/lib/pylib/urwid-2.1.0-py3.7-linux-x86_64.egg', '/home/scratch.edwardc_gpu_2/lab/tools/anaconda3/envs/d2l/lib/python38.zip', '/home/scratch.edwardc_gpu_2/lab/tools/anaconda3/envs/d2l/lib/python3.8', '/home/scratch.edwardc_gpu_2/lab/tools/anaconda3/envs/d2l/lib/python3.8/lib-dynload', '', '/home/edwardc/.local/lib/python3.8/site-packages', '/home/scratch.edwardc_gpu_2/lab/tools/anaconda3/envs/d2l/lib/python3.8/site-packages', '/home/scratch.edwardc_gpu_2/lab/tools/anaconda3/envs/d2l/lib/python3.8/site-packages/IPython/extensions', '/home/edwardc/.ipython']


In [23]:
# module only have one object instance during the whole runtime no matter how many places that import it.
# module a, b below both imported and changed var in module modsvar.
# they take effect in the same object instance of module modsvar.
# run below code again you will find that the result changes: module's code are executed only once at the first time it's imported

from testmods import modsvar
print (modsvar.var, id(modsvar))
from testmods import moda
print (modsvar.var)
from testmods import modb
print (modsvar.var)

modb 140737352981456
modb
modb


----

In [26]:
# double underscores variable

class A:
    @property
    def A_var(self):
        return self.__var
    
    @A_var.setter
    def A_var(self, value):
        self.__var = value
      
    
class B(A):
    @property
    def B_var(self):
        return self.__var
    
    @B_var.setter
    def B_var(self, value):
        self.__var = value

        
b_inst = B()

b_inst.B_var = 'B'
b_inst.A_var = 'A'

print(b_inst.B_var)
print(b_inst.A_var)

B
A


----

#### [super()](https://docs.python.org/3/library/functions.html#super)
> Reference: https://zhuanlan.zhihu.com/p/151856162 </br>
> More: [Super Considered Harmful](https://fuhm.net/super-harmful)

In [24]:
# https://docs.python.org/3.10/library/functions.html#super
def get_mro_info(t, o_or_t):
    if isinstance(o_or_t, t):
        mro = f"mro of instance: {o_or_t.__class__.mro()}"
    elif issubclass(o_or_t, t):
        mro = f"mro of class: {o_or_t.mro()}"
    else:
        raise AttributeError
    return mro

In [3]:
# example w/o super

class Base(object):
    def __init__(self):
        print("enter Base")
        print("leave Base")


class A(Base):
    def __init__(self):
        print("enter A")
        Base.__init__(self) #调用父类的构造函数进行初始化
        print("leave A")


class B(Base):
    def __init__(self):
        print("enter B")
        Base.__init__(self) #调用父类的构造函数进行初始化
        print("leave B")

class C(A,B):
    def __init__(self):
        print("enter C")
        A.__init__(self) #调用父类A的构造函数进行初始化
        B.__init__(self) #调用父类B的构造函数进行初始化
        print("leave C")

c=C()

## NOTE: 基类Base的构造函数被调用了两次，这是有问题的，正常的应该是：A的构造函数调用一次，B的构造函数调用一次，基类Base的构造函数调用一次。

enter C
enter A
enter Base
leave Base
leave A
enter B
enter Base
leave Base
leave B
leave C


In [27]:
# call super w/o two arguments, approach 0

# https://stackoverflow.com/questions/13126727/how-is-super-in-python-3-implemented

# super makes an implicit reference to a "magic" `__class__` name which behaves as a cell variable in the namespace of each class method.
# From the Python datamodel:
#   __class__ is an implicit closure reference created by the compiler if any methods in a class body refer to either __class__ or super.
#   This allows the zero argument form of super() to correctly identify the class being defined based on lexical scoping,
#   while the class or instance that was used to make the current call is identified based on the first argument passed to the method.
#   https://peps.python.org/pep-0367

class Base(object):
    def __init__(self):
        print(f"enter Base | {get_mro_info(__class__, self)}")
        super().__init__() #调用父类的构造函数进行初始化
        print("leave Base")

class A(object):
    def __init__(self):
        print(f"enter A | {get_mro_info(__class__, self)}")
        super().__init__() #调用父类的构造函数进行初始化
        print("leave A")

class B(Base):
    def __init__(self):
        print(f"enter B | {get_mro_info(__class__, self)}")
        super().__init__() #调用父类的构造函数进行初始化
        print("leave B")

class C(A,B):
    def __init__(self):
        print(f"enter C | {get_mro_info(__class__, self)}")
        super().__init__()
        print("leave C")

c=C()

enter C | mro of instance: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
enter A | mro of instance: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
enter B | mro of instance: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
enter Base | mro of instance: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
leave Base
leave B
leave A
leave C


In [36]:
# call super w/ two arguments, approach 1

class Base(object):
    def __init__(self):
        print(f"enter Base | {get_mro_info(__class__, self)}")
        super(__class__, self).__init__() #调用父类的构造函数进行初始化
        print("leave Base")

class A(object):
    def __init__(self):
        print(f"enter A | {get_mro_info(__class__, self)}")
        super(__class__, self).__init__() #调用父类的构造函数进行初始化
        print("leave A")

class B(Base):
    def __init__(self):
        print(f"enter B | {get_mro_info(__class__, self)}")
        super(__class__, self).__init__() #调用父类的构造函数进行初始化
        print("leave B")

class C(A,B):
    def __init__(self):
        print(f"enter C | {get_mro_info(__class__, self)}")
        super(__class__, self).__init__()
        print("leave C")

c=C()

enter C | mro of instance: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
enter A | mro of instance: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
enter B | mro of instance: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
enter Base | mro of instance: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
leave Base
leave B
leave A
leave C


In [37]:
# call super w/ two arguments, approach 2

class Base(object):
    def __init__(self):
        print(f"enter Base | {get_mro_info(__class__, self.__class__)}")
        super(__class__, self.__class__).__init__(self) #调用父类的构造函数进行初始化
        print("leave Base")

class A(object):
    def __init__(self):
        print(f"enter A | {get_mro_info(__class__, self.__class__)}")
        super(__class__, self.__class__).__init__(self) #调用父类的构造函数进行初始化
        print("leave A")

class B(Base):
    def __init__(self):
        print(f"enter B | {get_mro_info(__class__, self.__class__)}")
        super(__class__, self.__class__).__init__(self) #调用父类的构造函数进行初始化
        print("leave B")

class C(A,B):
    def __init__(self):
        print(f"enter C | {get_mro_info(__class__, self.__class__)}")
        super(__class__, self.__class__).__init__(self)
        print("leave C")

c=C()

enter C | mro of class: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
enter A | mro of class: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
enter B | mro of class: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
enter Base | mro of class: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
leave Base
leave B
leave A
leave C


In [9]:
# Skipping the super init intentionally will break initialization sequence

class Base(object):
    def __init__(self):
        print("enter Base")
        super().__init__() #调用父类的构造函数进行初始化
        print("leave Base")

class A(object):
    def __init__(self):
        print("enter A")
       #super().__init__() #调用父类的构造函数进行初始化
        print("leave A")

class B(Base):
    def __init__(self):
        print("enter B")
        super().__init__() #调用父类的构造函数进行初始化
        print("leave B")

class C(A,B):
    def __init__(self):
        print("enter C")
        super().__init__()
        print("leave C")

c=C()

enter C
enter A
leave A
leave C


In [25]:
class Base(object):
    def __new__(cls):
        print(f"enter Base | {get_mro_info(__class__, cls)}")
        super().__new__(cls)
        print("leave Base")

class A(object):
    def __new__(cls):
        print(f"enter A | {get_mro_info(__class__, cls)}")
        super().__new__(cls)
        print("leave A")

class B(Base):
    def __new__(cls):
        print(f"enter B | {get_mro_info(__class__, cls)}")
        super().__new__(cls)
        print("leave B")

class C(A,B):
    def __new__(cls):
        print(f"enter C | {get_mro_info(__class__, cls)}")
        super().__new__(cls)
        print("leave C")

c=C()

enter C | mro of class: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
enter A | mro of class: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
enter B | mro of class: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
enter Base | mro of class: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
leave Base
leave B
leave A
leave C


In [26]:
# equivalence to code section above

class Base(object):
    def __new__(cls):
        print(f"enter Base | {get_mro_info(__class__, cls)}")
        super(__class__, cls).__new__(cls)
        print("leave Base")

class A(object):
    def __new__(cls):
        print(f"enter A | {get_mro_info(__class__, cls)}")
        super(__class__, cls).__new__(cls)
        print("leave A")

class B(Base):
    def __new__(cls):
        print(f"enter B | {get_mro_info(__class__, cls)}")
        super(__class__, cls).__new__(cls)
        print("leave B")

class C(A,B):
    def __new__(cls):
        print(f"enter C | {get_mro_info(__class__, cls)}")
        super(__class__, cls).__new__(cls)
        print("leave C")

c=C()

enter C | mro of class: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
enter A | mro of class: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
enter B | mro of class: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
enter Base | mro of class: [<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
leave Base
leave B
leave A
leave C


----

#### The implementation of `object.__new__(...) && object.__init__(...)`

----

#### dict setdefault

In [3]:
rules = {'l':[1,2,3], 's':[2,3,4]}
rules.setdefault('l', [1])

[1, 2, 3]

In [16]:
import itertools
temp_str1 = ('abcdefg', 'tesdf')
temp_str2 = 'sadfsdafasdfasf'
print(list(itertools.chain(temp_str1, temp_str2)))
        

['abcdefg', 'tesdf', 's', 'a', 'd', 'f', 's', 'd', 'a', 'f', 'a', 's', 'd', 'f', 'a', 's', 'f']
